In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import explode
import os
spark=(
    SparkSession.builder
    .master('local[3]')
    .appName('transformApp')
    .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 12:06:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

df_boxoffice=spark.read.json('collection_parsed_2024116.json',multiLine=True)
df_movie_raw=spark.read.json('movies_parsed_2024116.json',multiLine=True)
df_filmmaker_raw=spark.read.json('filmmaker_parsed_2024116.json',multiLine=True)
df_genre_raw=spark.read.json('genre_parsed_2024116.json',multiLine=True)
df_distributor_raw=spark.read.json('distributor_parsed_2024116.json',multiLine=True)


In [3]:
df_movie=(
    df_movie_raw.select(explode(df_movie_raw.content).alias('movies'),'extracted_at','run_date')
        .select('movies.*','extracted_at','run_date')
        
)
df_genre=(
    df_genre_raw.select(explode(df_genre_raw.content).alias('genre'),'extracted_at','run_date')
        .select('genre.*','extracted_at','run_date')
        

)

df_filmmaker=(
    df_filmmaker_raw.select(explode(df_filmmaker_raw.content).alias('filmmaker'),'extracted_at','run_date')
        .select('filmmaker.*','extracted_at','run_date')
        
)

df_distributor=(
    df_distributor_raw.select(explode(df_distributor_raw.content).alias('distributor'),'extracted_at','run_date')
        .select('distributor.*','extracted_at','run_date')
        
)


In [4]:
# Bridge table b/w movies and genre dimension
bridge_movie_genre=df_genre.select('genre_id','movie_id')


# Bridge table b/w movies and filmmaker dimension
bridge_filmmaker_movie=(
    df_movie.withColumn('filmmaker',explode(df_movie['filmmakers']))
        .select('filmmaker','movie_id')
        .select('filmmaker.filmmaker_id','filmmaker.role','movie_id')
        
)

In [5]:
# Removing the duplicate values from dim_genre and dim_filmmaker which got created due to the many to many relationship b/w (genre,filmmaker) and movies
# Also dropping the movie_id as it does not match the dimension model schema for the corresponding dims
df_genre=(
    df_genre.dropDuplicates(['genre_id'])
            .drop('movie_id')
)
df_filmmaker=(
    df_filmmaker.dropDuplicates(['filmmaker_id'])
                .drop('movie_id')
)

In [ ]:
df_movie=(
    df_movie.withColumns({
    "release_date_id":df_movie.release_date_id.cast(IntegerType()),
    "duration":df_movie.duration.cast(IntegerType()),
    "widest_release":df_movie.widest_release.cast(IntegerType()),
    "imdb_rating":df_movie.imdb_rating.cast(FloatType()),
    "num_of_rating":df_movie.num_of_rating.cast(IntegerType()),
    "extracted_at":df_movie.extracted_at.cast(IntegerType()),
    "run_date":df_movie.run_date.cast(IntegerType())
    })
)

df_genre=(
    df_genre.withColumns({
        "total_movies":df_genre.total_movies.cast(IntegerType()),
        "extracted_at":df_genre.extracted_at.cast(IntegerType()),
        "run_date":df_genre.run_date.cast(IntegerType())
    })
)

df_filmmaker=(
    df_filmmaker.withColumns({
        "dob":df_filmmaker.dob.cast(DateType()),
        "total_movies":df_filmmaker.total_movies.cast(IntegerType()),
        "extracted_at":df_filmmaker.extracted_at.cast(IntegerType()),
        "run_date":df_filmmaker.run_date.cast(IntegerType())
    })
)

df_distributor=(
    df_distributor.withColumns({
        "total_movies":df_distributor.total_movies.cast(IntegerType()),
        "extracted_at":df_distributor.extracted_at.cast(IntegerType()),
        "run_date":df_distributor.run_date.cast(IntegerType())
    })
)

df_boxoffice=(
    df_boxoffice.withColumns({
        "run_date":df_boxoffice.run_date.cast(IntegerType()),
        "collection_domestic":df_boxoffice.collection_domestic.cast(FloatType()),
        "days_post_release":df_boxoffice.days_post_release.cast(IntegerType()),
        "num_of_releases":df_boxoffice.num_of_releases.cast(IntegerType()),
        "rank":df_boxoffice.rank.cast(IntegerType()),
        "num_theatres":df_boxoffice.num_theatres.cast(IntegerType()),
        "extracted_at":df_boxoffice.extracted_at.cast(IntegerType())
    })
)




In [ ]:
#TODO
    # 1. Add fields listed in the data model to the corresponding DataFrames
    # 2. Check for possible undesired values in every DataFrames' fields and fix it by replacing it with the desired value so everything can be casted properly.